In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
###############################################################

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
res='1km';t_res='5min'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(PlottingFunctions, inspect.isfunction)]
# functions

In [ ]:
########################################################
#PLOTTING
plotting=True

In [ ]:
# #DOMAIN SUBSETTING
# ocean_percent=2/8

# left_to_coast=data['xh'][0]+(data['xh'][-1]-data['xh'][0])*ocean_percent
# where_coast_xh=np.where(data['xh']>=left_to_coast)[0][0]#-25
# where_coast_xf=np.where(data['xf']>=left_to_coast)[0][0]#-25
# end_xh=len(data['xh'])-1-50
# end_xf=len(data['xf'])-1-50

# print(f'x in {0}:{where_coast_xh-1} FOR SEA')
# print(f'x in {where_coast_xh}:{end_xh} FOR LAND')
# # t_end=78 
# # if res=='250m':t_end=410
# # print(f't in {0}:{t_end} (6.5 hours)')
# t_start=36 
# print(f't in {t_start}:end (8 hours)')

# profile_array_e_g=profile_array_e_g[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]
# profile_array_d_g=profile_array_d_g[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]
# profile_array_e_c=profile_array_e_c[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]
# profile_array_d_c=profile_array_d_c[slice(0,78+1),:,:,slice(where_coast_xh,end_xh+1)]

In [ ]:
#constants
Cp=1004 #Jkg-1K-1
Cv=717 #Jkg-1K-1
Rd=Cp-Cv #Jkg-1K-1
eps=0.608

Lx=(data['xf'][-1].item()-data['xf'][0].item())*1000 #x length (m)
Ly=(data['yf'][-1].item()-data['yf'][0].item())*1000 #y length (m)
Np=len(parcel['xh']) #number of lagrangian parcles
dt=(data['time'][1]-data['time'][0]).item()/1e9 #sec
dx=(data['xf'][1].item()-data['xf'][0].item())*1e3 #meters
dy=(data['yf'][1].item()-data['yf'][0].item())*1e3 #meters
xs=data['xf'].values*1000
ys=data['yf'].values*1000
zs=data['zf'].values*1000

def zf(z):
    k=z #z is the # level of z
    out=data['zf'].values[k]*1000
    
    return out
# def rho(x,y,z,t):
#     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
#     p0=101325 #Pa
#     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
#     T=theta*(p/p0)**(Rd/Cp)
#     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
#     # Tv=T*(1+eps*qv)
#     Tv=T*(eps+qv)/(eps*(1+qv))
#     rho = p/(Rd*Tv)
#     out=rho
#     return out

def rho(x,y,z,rho_data_t):
    out=rho_data_t[z,y,x]
    return out
def m(t):
    rho_data_t=data['rho'].isel(time=t).data
    
    m=0
    #triple sum
    for k in range(len(data['zh'])):
        dz=(zf(k+1)-zf(k))
        for j in range(len(data['yh'])):
            for i in range(len(data['xh'])):
                rho_out=rho(i,j,k,rho_data_t)
                m+=rho_out*dz

    #triple sum
    out=m*dx*dy/Np
    return out

In [ ]:
if plotting==True:
    #Calculate Mass Constant
    # calculate='single_time'
    # calculate=True
    calculate=False
    
    if calculate==True:
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
        for t in np.arange(Nt):
            if np.mod(t,25)==0: print(t)
            m_arr[t]=m(t)
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    elif calculate=='single_time':
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
    
        t=0 #len(data['time'])//2 #Pick some middle time
        m_300=m(t)
        for t in np.arange(Nt):
            m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    else:
        dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/'
        m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')
    
    # # TESTING
    # lst=[]
    # for t in np.arange(133):
    #     lst.append(m_arr[t])
    
    # plt.plot(lst)
    # (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
# # NONOPTIMIZED LOADING AND AVERAGING (NOT RECOMMENDED)

# PROCESSING=False
# PROCESSING=True

# if PROCESSING==False:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
# if PROCESSING==True:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(dir3, "r") as h5f:
#     profile_array_e_g = h5f["profile_array_e_g"][:]
#     profile_array_e_c = h5f["profile_array_e_c"][:]
#     profile_array_d_g = h5f["profile_array_d_g"][:]
#     profile_array_d_c = h5f["profile_array_d_c"][:]

# def apply_constant(profile_array,apply):
#     if apply==True:
#         Nt=profile_array.shape[0]
#         Nz=profile_array.shape[1]
    
#         profile_array/=(dx*dy*dt)
#         for t in np.arange(Nt):
#             profile_array[t]*=m_arr[t]
#         for z in np.arange(Nz):
#             dz=zf(z+1)-zf(z)
#             profile_array[:,z]/=dz
#     return profile_array

# #APPLY CONSTANTS TO ENTRAINMENT VALUE
# ##################################################
# profile_array_e_g=apply_constant(profile_array_e_g,apply=True)
# profile_array_e_c=apply_constant(profile_array_e_c,apply=True)
# profile_array_d_g=-apply_constant(profile_array_d_g,apply=True)
# profile_array_d_c=-apply_constant(profile_array_d_c,apply=True)
# ##################################################

# # type='general'
# type='cloudy'

# if type=='general':
#     profile_array_e=profile_array_e_g
#     profile_array_d=profile_array_d_g
#     profile_array_net=profile_array_e-profile_array_d
# if type=='cloudy':
#     profile_array_e=profile_array_e_c
#     profile_array_d=profile_array_d_c
#     profile_array_net=profile_array_e-profile_array_d

In [ ]:
# #OPTIMIZED LOADING AND AVERAGING
# def apply_constant_tbyt(profile_array,t,apply):
#     if apply==True:
#         Nt=len(data['time'])
#         Nz=len(data['zh'])
    
#         profile_array/=(dx*dy*dt)
#         profile_array*=m_arr[t]
#         for z in np.arange(Nz):
#             dz=zf(z+1)-zf(z)
#             profile_array[z]/=dz
#     return profile_array


# PROCESSING=False
# PROCESSING=True

# if PROCESSING==False:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
# if PROCESSING==True:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

# def load_get_mean(e_string,d_string,type):
#     Nt=len(data['time']); Nz=len(data['zh']); 
#     e_output_array = np.zeros((Nt, Nz))
#     d_output_array = np.zeros((Nt, Nz))
#     net_output_array = np.zeros((Nt, Nz))

    
#     with h5py.File(dir3, "r") as h5f:
#         #Reading

#         for t in np.arange(Nt):
#             print(t)
#             profile_array_e = h5f[e_string][t]
#             profile_array_d = h5f[d_string][t]
    
#             #Applying Constants
#             profile_array_e=apply_constant_tbyt(profile_array_e,t,apply=True)
#             profile_array_d=-apply_constant_tbyt(profile_array_d,t,apply=True)
    
#             profile_array_net=profile_array_e-profile_array_d

#             e_mean_yx=np.mean(profile_array_e, axis = (1,2))
#             d_mean_yx=np.mean(profile_array_d, axis = (1,2))
#             net_mean_yx=np.mean(profile_array_net, axis = (1,2))

#             e_output_array[t]=e_mean_yx
#             d_output_array[t]=d_mean_yx
#             net_output_array[t]=net_mean_yx

    
#     return e_output_array, d_output_array, net_output_array

# # #TESTING
# # test=np.random.random((2,4,4,4))
# # one=np.mean(test[0],axis=(1,2))
# # two=np.mean(test[1],axis=(1,2))
# # full=np.mean(test,axis=(2,3))
# # print(full[0]==one)
# # print(full[1]==two)

# type='general'
# type='cloudy'

# if type=='general':
#     e_string="profile_array_e_g"
#     d_string="profile_array_d_g"
#     [profile_array_e,profile_array_d,profile_array_net] = load_get_mean(e_string,d_string,type)
    
# if type=='cloudy':
#     e_string="profile_array_e_c"
#     d_string="profile_array_d_c"
#     [profile_array_e,profile_array_d,profile_array_net] = load_get_mean(e_string,d_string,type)

In [ ]:
PROCESSING=False
# PROCESSING=True

if PROCESSING==False:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_combined_profiles_{res}_{t_res}_{Np_str}.h5'
if PROCESSING==True:
    dir3=dir+f'Project_Algorithms/Entrainment/OUTPUT/3D_entrainmentdetrainment_combined_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

import dask
import dask.array as da
from dask.diagnostics import ProgressBar

# Open the dataset with chunking
ds = xr.open_dataset(
    dir3,
    engine='h5netcdf',  # Use the correct engine based on the file type
    phony_dims='sort',
    chunks = {'phony_dim_0': 100,
              'phony_dim_1': 34,
              'phony_dim_2': 100,
              'phony_dim_3': 64#128
}
)

# Rename the dimensions
ds = ds.rename({
    'phony_dim_0': 't',   # Rename phony_dim_0 to 't'
    'phony_dim_1': 'z',   # Rename phony_dim_1 to 'z'
    'phony_dim_2': 'y',   # Rename phony_dim_2 to 'y'
    'phony_dim_3': 'x'    # Rename phony_dim_3 to 'x'
})

In [ ]:
#EVEN MORE OPTIMIZED WITH DASK

def apply_constant(profile_array, apply, dx, dy, dt, m_arr, dz):
    if apply:
        # Step 1: Divide by dx * dy * dt (scalar)
        profile_array = profile_array / (dx * dy * dt)

        # Step 2: Multiply by m_arr[t] — shape (Nt,) needs to broadcast to (Nt, Nz, Ny, Nx)
        m_arr_broadcasted = da.asarray(m_arr)[:, None, None, None]  # Shape: (Nt, 1, 1, 1)
        profile_array = profile_array * m_arr_broadcasted

        # Step 3: Divide by dz[z] — zf of shape (Nz + 1,), so dz is (Nz,)
        dz = da.asarray(dz)  # Shape: (Nz,)
        dz_broadcasted = dz[None, :, None, None]  # Shape: (1, Nz, 1, 1)
        profile_array = profile_array / dz_broadcasted

    return profile_array

profile_vars = ['c_to_g','g_to_c']
tz_profiles = {}
zf2=data['zf'].data*1000
dz=zf2[1:]-zf2[:-1]

with ProgressBar():
    for var in profile_vars:
        profile_array = ds[f'profile_array_{var}']
        # Apply the same transformation to each variable
        profile_array_updated = apply_constant(profile_array, apply=True, dx=dx, dy=dy, dt=dt, m_arr=m_arr, dz=dz)
        # Compute the zx mean
        tz_profile = profile_array_updated.mean(dim=('y', 'x'))
        tz_profiles[f'tz_profile_array_{var}'] = tz_profile#.compute()

    # Persist the results (keep them in memory across workers)
    tz_profiles_persisted = {k: v.persist() for k, v in tz_profiles.items()}
 
    # Compute all profiles at once to avoid individual .compute() calls
    computed_profiles = dask.compute(*tz_profiles_persisted.values())

    # Map the computed results back to the dictionary
    tz_profiles = dict(zip(tz_profiles_persisted.keys(), computed_profiles))


In [ ]:
#SAVING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Entrainment_tz_combined_profiles_{res}_{t_res}_{Np_str}.npz'
np.savez(dir3, **tz_profiles)

In [ ]:
########################################################################

In [ ]:
#LOADING
dir3=dir2+f'Project_Algorithms/Entrainment/OUTPUT/Entrainment_tz_combined_profiles_{res}_{t_res}_{Np_str}.npz'
tz_profiles=np.load(dir3)

In [ ]:
type1='general'
type1='cloudy'
tz_profile_array_c_to_g = tz_profiles['tz_profile_array_c_to_g'].copy()
tz_profile_array_g_to_c = tz_profiles['tz_profile_array_g_to_c'].copy()

In [ ]:
#Entrainment

#Plotting
############################################################
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

fig = plt.figure(figsize=(10, 8))
gs = GridSpec(2, 2, figure=fig)

######
cmap1 = plt.cm.viridis
cmap2 = plt.cm.seismic 
n_levels=29
######

######
vmax_shared = np.max([np.max(tz_profile_array_c_to_g), np.max(tz_profile_array_g_to_c)])
print(np.max([np.max(tz_profile_array_c_to_g), np.max(tz_profile_array_g_to_c)]))
norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
######

# First subplot: Entrainment
########################################
ax1 = fig.add_subplot(gs[0, 0])
contour1 = ax1.contourf(tz_profile_array_c_to_g.T, cmap=cmap1)
# contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
cbar1=fig.colorbar(contour1, ax=ax1)
Nz = len(data['zh'])
ax1.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax1.set_ylabel('z (km)');ax1.set_xlabel('t (timesteps)')
ax1.set_title('Cloudy to General Entrainment')

# Second subplot: Detrainment
########################################
ax2 = fig.add_subplot(gs[0, 1])
contour2 = ax2.contourf(tz_profile_array_g_to_c.T, cmap=cmap1)
# contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1, norm=norm_shared, levels=n_levels)
cbar2 = fig.colorbar(contour2, ax=ax2)
ax2.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax2.set_ylabel('z (km)');ax2.set_xlabel('t (timesteps)')
ax2.set_title('General to Cloudy Entrainment')

#FIXING SCIENTIFIC NOTATION

def apply_scientific_notation_colorbar(cbars):
    from matplotlib.ticker import ScalarFormatter
    formatter = ScalarFormatter(useMathText=True)
    formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
    for cbar in cbars:  # These must be Colorbar instances
        cbar.formatter = formatter
        cbar.update_ticks()
apply_scientific_notation_colorbar([cbar1,cbar2])

# Display the plot
plt.tight_layout()

#TESTING
print(f"Max of profile_array_c_to_g: {np.max(tz_profile_array_c_to_g)}")
print(f"Max of profile_array_g_to_c: {np.max(tz_profile_array_g_to_c)}")

###################### FIXING Y TICKS
Nz = len(data['zh'])
step = 4  # change to 2, 5, etc. depending on how spaced you want them
ytick_pos = np.arange(0, Nz, step)
ytick_labels = np.round(data['zf'].values[ytick_pos], 2)
for axis in [ax1,ax2]:#,ax3]:
    axis.set_yticks(ytick_pos)
    axis.set_yticklabels(ytick_labels, fontsize=8, rotation=0)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import numpy as np

if plotting == True:
    c_to_g = np.mean(tz_profile_array_c_to_g, axis=0)
    g_to_c = np.mean(tz_profile_array_g_to_c, axis=0)

    # Create a 2-row layout: top row with 2 plots, bottom row with 1 plot
    fig = plt.figure(figsize=(12, 10))
    gs = fig.add_gridspec(2, 2, height_ratios=[1, 1])

    ax1 = fig.add_subplot(gs[0, 0])  # Cloudy to General
    ax2 = fig.add_subplot(gs[0, 1])  # General to Cloudy
    ax3 = fig.add_subplot(gs[1, 0])  # Difference plot spans both columns

    # First subplot: cloudy to general
    ax1.plot(c_to_g, data['zh'], color='blue', label='cloudy to general')
    ax1.axvline(0, color='black')
    ax1.set_title('Cloudy to General')
    ax1.set_ylabel('Height (m)')
    ax1.legend()
    ax1.xaxis.set_major_formatter(ScalarFormatter(useMathText=True, useOffset=False))
    ax1.xaxis.get_major_formatter().set_scientific(True)
    ax1.xaxis.get_major_formatter().set_powerlimits((-1, 1))

    # Second subplot: general to cloudy
    ax2.plot(g_to_c, data['zh'], color='red', label='general to cloudy')
    ax2.axvline(0, color='black')
    ax2.set_title('General to Cloudy')
    ax2.legend()
    ax2.xaxis.set_major_formatter(ScalarFormatter(useMathText=True, useOffset=False))
    ax2.xaxis.get_major_formatter().set_scientific(True)
    ax2.xaxis.get_major_formatter().set_powerlimits((-1, 1))

    # Third subplot: difference
    ax3.plot(c_to_g - g_to_c, data['zh'], label='c→g minus g→c', color='green')
    ax3.axvline(0, color='black')
    ax3.axhline(1.24, color='purple', linestyle='dashed', label='avg cloud base')
    ax3.set_title('Difference: Cloudy→General minus General→Cloudy')
    ax3.set_ylabel('Height (m)')
    ax3.legend()
    ax3.xaxis.set_major_formatter(ScalarFormatter(useMathText=True, useOffset=False))
    ax3.xaxis.get_major_formatter().set_scientific(True)
    ax3.xaxis.get_major_formatter().set_powerlimits((-1, 1))

    fig.suptitle('2D Entrainment Transfer Analysis')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


In [ ]:
##################################
#Z-X Plot

In [ ]:
import xarray as xr
import dask.array as da

def apply_constant(profile_array, apply, dx, dy, dt, m_arr, dz):
    if apply:
        # Step 1: Divide by dx * dy * dt (scalar)
        profile_array = profile_array / (dx * dy * dt)

        # Step 2: Multiply by m_arr[t] — shape (Nt,) needs to broadcast to (Nt, Nz, Ny, Nx)
        m_arr_broadcasted = da.asarray(m_arr)[:, None, None, None]  # Shape: (Nt, 1, 1, 1)
        profile_array = profile_array * m_arr_broadcasted

        # Step 3: Divide by dz[z] — zf of shape (Nz + 1,), so dz is (Nz,)
        dz = da.asarray(dz)  # Shape: (Nz,)
        dz_broadcasted = dz[None, :, None, None]  # Shape: (1, Nz, 1, 1)
        profile_array = profile_array / dz_broadcasted

    return profile_array

profile_vars = ['c_to_g','g_to_c']
zx_profiles = {}
zf2=data['zf'].data*1000
dz=zf2[1:]-zf2[:-1]

with ProgressBar():
    for var in profile_vars:
        profile_array = ds[f'profile_array_{var}']
        # Apply the same transformation to each variable
        profile_array_updated = apply_constant(profile_array, apply=True, dx=dx, dy=dy, dt=dt, m_arr=m_arr, dz=dz)
        # Compute the zx mean
        zx_profile = profile_array_updated.mean(dim=('t', 'y'))
        zx_profiles[f'zx_profile_array_{var}'] = zx_profile#.compute()

    # Persist the results (keep them in memory across workers)
    zx_profiles_persisted = {k: v.persist() for k, v in zx_profiles.items()}
 
    # Compute all profiles at once to avoid individual .compute() calls
    computed_profiles = dask.compute(*zx_profiles_persisted.values())

    # Map the computed results back to the dictionary
    tz_profiles = dict(zip(zx_profiles_persisted.keys(), computed_profiles))

In [ ]:
#SAVING
dir3=dir2+f'Project_Algorithms/Entrainment/Entrainment_zx_combined_profiles_{res}_{t_res}_{Np_str}.npz'
np.savez(dir3, **tz_profiles)

In [ ]:
##########################################

In [ ]:
#LOADING
dir3=dir2+f'Project_Algorithms/Entrainment/Entrainment_zx_combined_profiles_{res}_{t_res}_{Np_str}.npz'
zx_profiles=np.load(dir3)

In [ ]:
type1='general'
type1='cloudy'
zx_profile_array_c_to_g = tz_profiles['zx_profile_array_c_to_g'].copy()
zx_profile_array_g_to_c = tz_profiles['zx_profile_array_g_to_c'].copy()

In [ ]:
#Entrainment

#Plotting
############################################################
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

fig = plt.figure(figsize=(10, 8))
gs = GridSpec(2, 2, figure=fig)

######
cmap1 = plt.cm.viridis
cmap2 = plt.cm.seismic 
n_levels=29
######

######
vmax_shared = np.max([np.max(zx_profile_array_c_to_g), np.max(zx_profile_array_g_to_c)])
print(np.max([np.max(zx_profile_array_c_to_g), np.max(zx_profile_array_g_to_c)]))
norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
######

# First subplot: Entrainment
########################################
ax1 = fig.add_subplot(gs[0, 0])
# contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1)
contour1 = ax1.contourf(zx_profile_array_c_to_g, cmap=cmap1, norm=norm_shared, levels=n_levels)
cbar1=fig.colorbar(contour1, ax=ax1)
Nz = len(data['zh'])
ax1.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax1.set_ylabel('z (km)');ax1.set_xlabel('x (km)')
ax1.set_title('Entrainment using Lagrangian Binary Array',fontsize=8)

# Second subplot: Detrainment
########################################
ax2 = fig.add_subplot(gs[0, 1])
# contour2 = ax2.contourf(profile_array_d.T, cmap=cmap1)
contour2 = ax2.contourf(zx_profile_array_g_to_c, cmap=cmap1, norm=norm_shared, levels=n_levels)
cbar2 = fig.colorbar(contour2, ax=ax2)
ax2.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax2.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax2.set_ylabel('z (km)');ax2.set_xlabel('x (km)')
ax2.set_title('Detrainment')

#FIXING YTICKS
ax3.set_yticks(np.arange(Nz))
new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
ax3.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
ax3.set_ylabel('z (km)');ax3.set_xlabel('x (km)')
ax3.set_title('Entrainment - Detrainment')

#FIXING XTICKS
fix_tick_labels([ax1,ax2,ax3], data, data_dim='x', tick_axis='x', d_xtick=85, d_ytick=20, cell_loc='center',round=1,meters=False) 


#FIXING SCIENTIFIC NOTATION

def apply_scientific_notation_colorbar(cbars):
    from matplotlib.ticker import ScalarFormatter
    formatter = ScalarFormatter(useMathText=True)
    formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
    for cbar in cbars:  # These must be Colorbar instances
        cbar.formatter = formatter
        cbar.update_ticks()
apply_scientific_notation_colorbar([cbar1,cbar2])

# Display the plot
plt.tight_layout()